# **사용법**
현재 이 파일은 깃허브에서 바로 필요화일을 구글 코랩에 로드하고 구글 드라이브(자료 및 저장공간)을 이용해서 작동하도록 구성되어 있음. 따라서 이를 반영하여 앞부분을 조정하고 파일 패스를 지정하면 됨.

그리고 결과값을 요약하는 Optuna 보고서도 자동 생성되니 확인하셔서 파일 경로를 필요에 따라 지정하세요

예측 아웃풋은 predition.csv화일명으로 구글 드라이브에 나오게 현재는 되어 있으니 경로,이름, 포맷(현재는 1개 레이블(신뢰도)이 추가 되어있음다!)을 수정 반영하면 될 듯해요. 이후 블럭에 아웃풋을 분석하는 별도 코드 블럭들이 되어 있어요. 아웃풋을 바로 확인하시길!
  
이후 별도 추가 블럭에 아웃풋을 케글 포맷에 맞추어 자동으로 submission 요구형태로 변환하는 파일도 준비되어 있으니 패스를 지정해서 활용하시면 됩니다.

# **소개 및 요약**

BERT Model은 transformer모델에 pre-trained 모델로 boemi-kcbert base를 활용. Boemi-kcbert는 naver의 댓글을 베이스로 해서 학습된 모델로 비속어, 부정어 한글 분류에서 가장 성능이 좋다고 평가됨. 해당 모델로 실행 결과 일반 대화 분류 성과가 증가한 것이 관찰됨. 팀의 선택가능한 베이스라인 모델 중 하나라고 생각됨.

하이퍼파라미터는 optuna를 활용, 자동최적화를 도모했음. optuna 방식은 먼저 trial로 학습을 진행해서 그중 최적 하이퍼파라미터를 선정해서 최종 학습하는 방식으로 약간 더 추가적인 시간소요가 발생함. 결론적으로 하이퍼파라미터 변경만으로는 미미한 개선효과를 보였고 핵심적인 개선 영역은 아니라고 판단됨.

버트 모델에 대한 이해가 부족해서 클로드를 통해서 전체를 구성하고, gemini를 통해서 수정했음. 모델 개발을 우선적으로 완성하기 위하여, 모듈화된 팀의 코드는 사용하지 못했으므로 깊은 양해를 부탁합니다. 처음부터 구성한 상태임.

In [1]:
# 깃허브에서 코랩으로 자료를 바로 가져오도록 셋팅. 각 환경에 맞게 수정 요망
from getpass import getpass
token = getpass('GitHub token 입력: ')
!git clone https://{token}@github.com/hiyslee/DLthon_pepero_day.git

GitHub token 입력: ··········
Cloning into 'DLthon_pepero_day'...
remote: Enumerating objects: 309, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 309 (delta 7), reused 4 (delta 4), pack-reused 288 (from 2)
Receiving objects: 100% (309/309), 128.66 MiB | 20.71 MiB/s, done.
Resolving deltas: 100% (134/134), done.


In [2]:
#구글 드라이브를 저장소로 선택함
# ============================================================================
# 01. Google Drive 마운트
# ============================================================================
print("="*70)
print("01. Google Drive 마운트")
print("="*70)

from google.colab import drive
drive.mount('/content/drive')
print("✓ Google Drive 마운트 완료\n")


01. Google Drive 마운트
Mounted at /content/drive
✓ Google Drive 마운트 완료



In [3]:
# 트랜스포머 설치. 에러 생겨서 개선함
!pip install --upgrade pip && !pip install torch && !pip install transformers --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
/bin/bash: line 1: !pip: command not found


In [5]:
# beomi/kcbert-base 모델 - 완벽한 전처리 + 클래스 가중치 + 신뢰도 분류
"""
직장 내 대화 분류 - BERT 모델 (완전판)
Optuna 하이퍼파라미터 자동 튜닝 + 완벽한 전처리 + 클래스 불균형 해결

✨ 주요 기능:
  • 완벽한 데이터 전처리 (BOM 제거, 공백 정규화, 길이 필터링 등)
  • 클래스 가중치 자동 계산 및 적용 (불균형 해결)
  • Optuna를 이용한 자동 하이퍼파라미터 튜닝
  • 신뢰도 기반 분류 조정 (3단계)
    - High (≥0.85): 높은 신뢰도 - 예측 그대로 사용
    - Mid (0.6-0.85): 중간 신뢰도 - 예측 그대로 사용
    - Low (<0.6): 낮은 신뢰도 - "일반 대화"로 재분류
  • Top-K 예측 제공
  • 신뢰도 등급별 출력 파일 생성
  • 상세한 통계 분석 및 시각화

🔥 개선 사항:
  - 클래스 불균형 문제 해결 (가중치 적용)
  - 신뢰도 임계값 최적화 (0.8 → 0.6)
  - 텍스트 품질 향상 (완벽한 전처리)
  - 데이터 검증 강화
"""

# ============================================================================
# 01. 환경 설정 - GPU 메모리 최적화
# ============================================================================
import os

# ✅ PyTorch 메모리 할당 최적화
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# ✅ Transformers 캐시 비활성화 (선택사항)
os.environ['TRANSFORMERS_OFFLINE'] = '0'

print("="*70)
print("01. GPU 메모리 최적화 설정 완료")
print("="*70 + "\n")

# ============================================================================
# 02. 필요한 라이브러리 설치
# ============================================================================
print("="*70)
print("02. 라이브러리 설치")
print("="*70)

import subprocess
import sys

packages = ['torch', 'transformers', 'scikit-learn', 'tqdm', 'pandas', 'numpy', 'optuna', 'matplotlib']
for package in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from torch.optim import AdamW
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import optuna
from optuna.trial import TrialState
import time
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
import re

print("✓ 모든 라이브러리 설치 완료\n")

# ============================================================================
# 03. 기본 설정값
# ============================================================================
print("="*70)
print("03. 기본 설정")
print("="*70)

OUTPUT_PATH = '/content/drive/My Drive/predictions.csv'
TUNING_REPORT_PATH = '/content/drive/My Drive/optuna_tuning_report.txt'
TRAIN_PATH = '/content/DLthon_pepero_day/Data/aiffel-dl-thon-dktc-online-15/train.csv'
TEST_PATH = '/content/DLthon_pepero_day/Data/aiffel-dl-thon-dktc-online-15/test.csv'

MODEL_NAME = 'beomi/kcbert-base'
VALIDATION_RATIO = 0.2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Optuna 설정
N_TRIALS = 5
TIMEOUT_PER_TRIAL = 1800

# 🎯 신뢰도 기반 분류 설정 (3단계) - 최적화됨
CONFIDENCE_THRESHOLD_MID = 0.6   # Mid/Low 경계 (0.8 → 0.6으로 하향)
CONFIDENCE_THRESHOLD_HIGH = 0.85  # High/Mid 경계 (0.9 → 0.85로 하향)
TOP_K_PREDICTIONS = 3
USE_TOP_K = True

# 전처리 설정
MIN_TEXT_LENGTH = 10     # 최소 텍스트 길이
MAX_TEXT_LENGTH = 1500   # 최대 텍스트 길이 (토큰 제한 고려)

print(f"✓ 모델: {MODEL_NAME}")
print(f"✓ 디바이스: {DEVICE}")
print(f"✓ Optuna 시도 횟수: {N_TRIALS}")
print(f"✓ Validation 비율: {VALIDATION_RATIO*100}%")
print(f"\n✓ 신뢰도 임계값 (3단계) - 최적화:")
print(f"  • High: ≥ {CONFIDENCE_THRESHOLD_HIGH}")
print(f"  • Mid: {CONFIDENCE_THRESHOLD_MID} ~ {CONFIDENCE_THRESHOLD_HIGH}")
print(f"  • Low: < {CONFIDENCE_THRESHOLD_MID} → '일반 대화'로 재분류")
print(f"\n✓ 전처리 설정:")
print(f"  • 최소 길이: {MIN_TEXT_LENGTH}자")
print(f"  • 최대 길이: {MAX_TEXT_LENGTH}자\n")

# ✅ GPU 메모리 정보 출력
if torch.cuda.is_available():
    print(f"💾 GPU 정보:")
    print(f"  • GPU: {torch.cuda.get_device_name(0)}")
    print(f"  • 총 메모리: {torch.cuda.get_device_properties(0).total_memory/1024**3:.2f} GB")
    print()

# ============================================================================
# 04. 완벽한 전처리 함수들
# ============================================================================
print("="*70)
print("04. 전처리 함수 정의")
print("="*70)

def remove_bom(text):
    """BOM (Byte Order Mark) 제거"""
    if isinstance(text, str):
        return text.replace('\ufeff', '').replace('\ufffe', '')
    return text

def normalize_whitespace(text):
    """공백, 탭, 줄바꿈 정규화"""
    if not isinstance(text, str):
        return text

    # 탭을 공백으로
    text = text.replace('\t', ' ')

    # 연속된 공백을 하나로
    text = re.sub(r' +', ' ', text)

    # 연속된 줄바꿈을 최대 2개로 (대화 구조 유지)
    text = re.sub(r'\n\n+', '\n\n', text)

    # 앞뒤 공백 제거
    text = text.strip()

    return text

def normalize_quotes(text):
    """따옴표 정규화"""
    if not isinstance(text, str):
        return text

    # 다양한 따옴표를 표준 따옴표로
    quote_map = {
        '"': '"', '"': '"',  # 큰따옴표
        ''': "'", ''': "'",  # 작은따옴표
        '″': '"',  # 인치 기호
        '‶': '"',  # 역 인치 기호
    }

    for old, new in quote_map.items():
        text = text.replace(old, new)

    return text

def clean_special_chars(text):
    """제어 문자 제거"""
    if not isinstance(text, str):
        return text

    # 제어 문자 제거 (줄바꿈, 탭 제외)
    text = ''.join(char for char in text if ord(char) >= 32 or char in '\n\t')

    return text

def clean_text(text):
    """전체 텍스트 정제 파이프라인"""
    text = remove_bom(text)
    text = normalize_whitespace(text)
    text = normalize_quotes(text)
    text = clean_special_chars(text)
    return text

def is_valid_conversation(text, min_length=10, max_length=2000):
    """대화 유효성 검증"""
    if not isinstance(text, str):
        return False

    # 빈 문자열
    if not text or not text.strip():
        return False

    # 길이 체크
    if len(text) < min_length or len(text) > max_length:
        return False

    # 의미 있는 문자가 있는지
    if not re.search(r'[가-힣a-zA-Z0-9]', text):
        return False

    return True

def preprocess_dataframe(df, min_length=10, max_length=1500, verbose=True):
    """완벽한 전처리 파이프라인"""

    if verbose:
        print(f"\n🧹 데이터 전처리 시작...")
        print(f"  원본 데이터: {len(df):,}개")

    initial_count = len(df)
    stats = {'original': initial_count, 'steps': []}

    # 1. 필수 컬럼 확인
    required_columns = ['conversation', 'class']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"필수 컬럼 누락: {required_columns}")

    # 2. 결측치 제거
    before = len(df)
    df = df.dropna(subset=['conversation', 'class'])
    removed = before - len(df)
    if verbose and removed > 0:
        print(f"  ✓ 결측치 제거: {removed}개")
    stats['steps'].append(('결측치', removed))

    # 3. 빈 문자열 제거
    before = len(df)
    df = df[df['conversation'].str.strip().str.len() > 0]
    removed = before - len(df)
    if verbose and removed > 0:
        print(f"  ✓ 빈 문자열 제거: {removed}개")
    stats['steps'].append(('빈 문자열', removed))

    # 4. 텍스트 정제
    if verbose:
        print(f"  ✓ 텍스트 정제 중 (BOM, 공백, 따옴표, 특수문자)...")
    df['conversation'] = df['conversation'].apply(clean_text)
    df['class'] = df['class'].apply(lambda x: x.strip() if isinstance(x, str) else x)

    # 5. 길이 필터링
    before = len(df)
    df['_temp_length'] = df['conversation'].str.len()

    too_short = (df['_temp_length'] < min_length).sum()
    too_long = (df['_temp_length'] > max_length).sum()

    df = df[(df['_temp_length'] >= min_length) & (df['_temp_length'] <= max_length)]
    df = df.drop('_temp_length', axis=1)

    removed = before - len(df)
    if verbose and removed > 0:
        print(f"  ✓ 길이 필터링: {removed}개 제거 (짧음: {too_short}, 김: {too_long})")
    stats['steps'].append(('길이 필터링', removed))

    # 6. 유효성 검증
    before = len(df)
    df = df[df['conversation'].apply(lambda x: is_valid_conversation(x, min_length, max_length))]
    removed = before - len(df)
    if verbose and removed > 0:
        print(f"  ✓ 유효성 검증: {removed}개 제거")
    stats['steps'].append(('유효성', removed))

    # 7. 중복 제거
    before = len(df)
    df = df.drop_duplicates(subset=['conversation', 'class'])
    removed = before - len(df)
    if verbose and removed > 0:
        print(f"  ✓ 중복 제거: {removed}개")
    stats['steps'].append(('중복', removed))

    # 8. 인덱스 재설정
    df = df.reset_index(drop=True)

    stats['final'] = len(df)
    stats['total_removed'] = initial_count - len(df)
    stats['retention_rate'] = (len(df) / initial_count * 100) if initial_count > 0 else 0

    if verbose:
        print(f"\n  ✅ 전처리 완료!")
        print(f"  최종 데이터: {len(df):,}개 (유지율: {stats['retention_rate']:.1f}%)")
        total_removed = sum(count for _, count in stats['steps'])
        print(f"  총 제거: {total_removed}개\n")

    return df, stats

print("✓ 전처리 함수 정의 완료\n")

# ============================================================================
# 05. Dataset 클래스 정의
# ============================================================================
print("="*70)
print("05. Dataset 클래스 정의")
print("="*70)

class ConversationDataset(Dataset):
    def __init__(self, conversations, tokenizer, labels=None, max_length=512):
        self.conversations = conversations
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        text = self.conversations[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        item = {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0)
        }

        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)

        return item

print("✓ Dataset 클래스 정의 완료\n")

# ============================================================================
# 06. 학습 데이터 로드 및 완벽한 전처리
# ============================================================================
print("="*70)
print("06. 학습 데이터 로드 및 완벽한 전처리")
print("="*70)

df_train = pd.read_csv(TRAIN_PATH)
print(f"✓ 원본 데이터: {len(df_train):,}개")

# 완벽한 전처리 적용
df_train, preprocess_stats = preprocess_dataframe(
    df_train,
    min_length=MIN_TEXT_LENGTH,
    max_length=MAX_TEXT_LENGTH,
    verbose=True
)

print(f"📊 클래스 분포:")
class_distribution = df_train['class'].value_counts()
print(class_distribution)
print()

# 클래스 균형도 체크
max_count = class_distribution.max()
min_count = class_distribution.min()
imbalance_ratio = max_count / min_count

print(f"⚖️  클래스 균형도 분석:")
print(f"  최대 클래스: {max_count:,}개")
print(f"  최소 클래스: {min_count:,}개")
print(f"  불균형 비율: {imbalance_ratio:.2f}:1")

if imbalance_ratio > 3:
    print(f"  ⚠️ 심각한 불균형! 클래스 가중치 적용 필수")
elif imbalance_ratio > 2:
    print(f"  ⚠️ 불균형 있음. 클래스 가중치 적용 권장")
else:
    print(f"  ✅ 균형 잡힌 데이터셋")
print()

# ============================================================================
# 07. 라벨 인코딩
# ============================================================================
print("="*70)
print("07. 라벨 인코딩")
print("="*70)

label_encoder = LabelEncoder()
df_train['label'] = label_encoder.fit_transform(df_train['class'])

print(f"✓ 클래스 개수: {len(label_encoder.classes_)}")
print(f"\n클래스 매핑:")
for i, class_name in enumerate(label_encoder.classes_):
    count = (df_train['class'] == class_name).sum()
    percentage = count / len(df_train) * 100
    print(f"  {i}: {class_name:20s} - {count:5,}개 ({percentage:5.1f}%)")
print()

# ============================================================================
# 08. 클래스 가중치 계산 (중요!)
# ============================================================================
print("="*70)
print("08. 클래스 가중치 계산 (불균형 해결)")
print("="*70)

# 클래스 가중치 계산
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(df_train['label']),
    y=df_train['label']
)

# 딕셔너리로 변환
class_weights_dict = {i: weight for i, weight in enumerate(class_weights_array)}

# PyTorch 텐서로 변환
class_weights_tensor = torch.FloatTensor(class_weights_array).to(DEVICE)

print(f"✓ 클래스 가중치 계산 완료\n")
print(f"📊 클래스별 가중치:")
for i, (class_name, weight) in enumerate(zip(label_encoder.classes_, class_weights_array)):
    count = (df_train['label'] == i).sum()
    print(f"  {class_name:20s}: 가중치 {weight:.4f} ({count:,}개)")

print(f"\n💡 가중치 효과:")
print(f"  • 데이터가 적은 클래스 → 높은 가중치 (더 중요하게 학습)")
print(f"  • 데이터가 많은 클래스 → 낮은 가중치")
print(f"  • 불균형 문제 자동 보정!")
print()

# ============================================================================
# 09. 학습/검증 데이터 분할
# ============================================================================
print("="*70)
print("09. 학습/검증 데이터 분할")
print("="*70)

X_train, X_val, y_train, y_val = train_test_split(
    df_train['conversation'].values,
    df_train['label'].values,
    test_size=VALIDATION_RATIO,
    random_state=42,
    stratify=df_train['label'].values
)

print(f"✓ 학습 데이터: {len(X_train):,}개 ({(1-VALIDATION_RATIO)*100:.0f}%)")
print(f"✓ 검증 데이터: {len(X_val):,}개 ({VALIDATION_RATIO*100:.0f}%)\n")

# 데이터 품질 검사
print(f"🔍 데이터 품질 검사:")
print(f"  • 학습 라벨 범위: {y_train.min()} ~ {y_train.max()}")
print(f"  • 검증 라벨 범위: {y_val.min()} ~ {y_val.max()}")
print(f"  • 학습 라벨 유니크: {len(np.unique(y_train))}개")
print(f"  • 검증 라벨 유니크: {len(np.unique(y_val))}개")

# 샘플 확인
print(f"\n📝 학습 데이터 샘플 (처음 3개):")
for i in range(min(3, len(X_train))):
    print(f"  {i+1}. Text: {X_train[i][:60]}...")
    print(f"     Label: {y_train[i]} ({label_encoder.classes_[y_train[i]]})")

print()

# ============================================================================
# 10. 토크나이저 로드 및 호환성 체크
# ============================================================================
print("="*70)
print("10. 토크나이저 로드 및 호환성 체크")
print("="*70)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
config = AutoConfig.from_pretrained(MODEL_NAME)
model_max_length = config.max_position_embeddings

print(f"✓ 토크나이저 로드 완료")
print(f"✓ 모델 최대 위치 임베딩: {model_max_length}")
print(f"✓ 사용 가능한 최대 길이: {min(model_max_length, 512)}\n")

SAFE_MAX_LENGTH = min(model_max_length, 512)

# 토크나이저 호환성 체크 (샘플링)
print(f"🔧 토크나이저 호환성 체크 (샘플 1000개)...")
sample_size = min(1000, len(X_train))
sample_indices = np.random.choice(len(X_train), sample_size, replace=False)
sample_texts = X_train[sample_indices]

over_length_count = 0
token_lengths = []

for text in sample_texts:
    tokens = tokenizer.encode(text, add_special_tokens=True)
    token_lengths.append(len(tokens))
    if len(tokens) > SAFE_MAX_LENGTH:
        over_length_count += 1

avg_token_length = np.mean(token_lengths)
max_token_length = np.max(token_lengths)

print(f"  • 평균 토큰 길이: {avg_token_length:.1f}")
print(f"  • 최대 토큰 길이: {max_token_length}")
print(f"  • MAX_LENGTH({SAFE_MAX_LENGTH}) 초과: {over_length_count}개 ({over_length_count/sample_size*100:.1f}%)")

if over_length_count > sample_size * 0.1:
    print(f"  ⚠️ 10% 이상이 MAX_LENGTH 초과! 512 사용 권장")
elif over_length_count > 0:
    print(f"  ⚠️ 일부 데이터가 MAX_LENGTH 초과. 잘림 현상 발생 가능")
else:
    print(f"  ✅ 모든 데이터가 MAX_LENGTH 이내")
print()

# ============================================================================
# 11. 모델 로드 함수 (클래스 가중치 포함)
# ============================================================================
def create_model(num_labels):
    """모델 생성"""
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=num_labels
    ).to(DEVICE)
    return model

print("="*70)
print("11. 모델 로드 함수 정의 완료")
print("="*70 + "\n")

# ============================================================================
# 12. 검증 함수 (Optuna 시도용)
# ============================================================================
def evaluate_model(model, val_loader):
    """검증 데이터로 모델 평가 (Accuracy & F1 Score)"""
    model.eval()
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=1)

            val_preds.extend(predictions.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(val_labels, val_preds)
    f1 = f1_score(val_labels, val_preds, average='weighted', zero_division=0)

    return accuracy, f1

# ============================================================================
# 13. 목적 함수 (Optuna용) - 클래스 가중치 적용
# ============================================================================
def objective(trial):
    """Optuna 목적 함수 - 클래스 가중치 적용"""

    print(f"\n{'='*70}")
    print(f"Trial {trial.number + 1}")
    print(f"{'='*70}")

    torch.cuda.empty_cache()
    import gc
    gc.collect()

    # 하이퍼파라미터 제안
    batch_size = trial.suggest_categorical('batch_size', [8, 16])
    epochs = trial.suggest_int('epochs', 3, 5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 5e-5, log=True)
    max_length = trial.suggest_categorical('max_length', [256, 512])

    print(f"\n📊 제안된 하이퍼파라미터:")
    print(f"  • Batch Size: {batch_size}")
    print(f"  • Epochs: {epochs}")
    print(f"  • Learning Rate: {learning_rate:.2e}")
    print(f"  • Max Length: {max_length}")

    if torch.cuda.is_available():
        print(f"\n💾 GPU 메모리 (Trial 시작):")
        print(f"  • 할당됨: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
        print(f"  • 예약됨: {torch.cuda.memory_reserved(0)/1024**3:.2f} GB")

    try:
        # Dataset 생성
        train_dataset = ConversationDataset(
            X_train, tokenizer, labels=y_train, max_length=max_length
        )
        val_dataset = ConversationDataset(
            X_val, tokenizer, labels=y_val, max_length=max_length
        )

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        # 모델 생성
        model = create_model(len(label_encoder.classes_))
        optimizer = AdamW(model.parameters(), lr=learning_rate)

        # 손실 함수에 클래스 가중치 적용
        criterion = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)

        # 학습
        print(f"\n🔄 모델 학습 중 (클래스 가중치 적용)...")
        best_val_accuracy = 0
        best_val_f1 = 0

        for epoch in range(epochs):
            model.train()
            total_loss = 0
            train_correct = 0
            train_total = 0

            pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
            for batch_idx, batch in enumerate(pbar):
                input_ids = batch['input_ids'].to(DEVICE)
                attention_mask = batch['attention_mask'].to(DEVICE)
                labels = batch['labels'].to(DEVICE)

                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                )

                # 클래스 가중치가 적용된 손실 계산
                loss = criterion(outputs.logits, labels)

                if torch.isnan(loss):
                    print(f"\n⚠️ NaN loss 발견! Epoch {epoch+1}, Batch {batch_idx}")
                    raise ValueError("Loss became NaN")

                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                total_loss += loss.item()
                predictions = torch.argmax(outputs.logits, dim=1)
                train_correct += (predictions == labels).sum().item()
                train_total += labels.size(0)

                pbar.set_postfix({
                    'loss': f'{total_loss/(batch_idx+1):.4f}',
                    'acc': f'{train_correct/train_total:.4f}'
                })

            train_accuracy = train_correct / train_total
            avg_train_loss = total_loss / len(train_loader)

            # 검증
            val_accuracy, val_f1 = evaluate_model(model, val_loader)

            print(f"  Epoch {epoch+1}/{epochs}:")
            print(f"    Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.4f}")
            print(f"    Val Acc: {val_accuracy:.4f} | Val F1: {val_f1:.4f}")

            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                best_val_f1 = val_f1

            # 조기 종료
            trial.report(best_val_accuracy, epoch)
            if trial.should_prune():
                print(f"  ⚠️ Trial pruned at epoch {epoch+1}")
                raise optuna.TrialPruned()

        print(f"\n✅ Trial {trial.number + 1} 완료")
        print(f"   Best Val Accuracy: {best_val_accuracy:.4f} ({best_val_accuracy*100:.2f}%)")
        print(f"   Best Val F1: {best_val_f1:.4f}")

        return best_val_accuracy

    except optuna.TrialPruned:
        raise
    except Exception as e:
        print(f"\n❌ Trial {trial.number + 1} 실패: {str(e)}")
        import traceback
        print(f"상세 에러:\n{traceback.format_exc()}")
        return 0.0

    finally:
        print(f"\n🧹 GPU 메모리 정리 중...")
        try:
            del model
            del optimizer
            del criterion
            del train_loader
            del val_loader
            del train_dataset
            del val_dataset
        except:
            pass

        torch.cuda.empty_cache()
        gc.collect()

        if torch.cuda.is_available():
            print(f"💾 GPU 메모리 (Trial 종료):")
            print(f"  • 할당됨: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
            print(f"  • 예약됨: {torch.cuda.memory_reserved(0)/1024**3:.2f} GB")

print("="*70)
print("12. 목적 함수 정의 완료 (클래스 가중치 적용)")
print("="*70 + "\n")

# ============================================================================
# 14. Optuna 최적화 실행
# ============================================================================
print("="*70)
print("13. Optuna 하이퍼파라미터 튜닝 시작")
print("="*70)

sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(
    direction='maximize',
    sampler=sampler,
    pruner=optuna.pruners.MedianPruner()
)

start_tuning_time = time.time()
study.optimize(objective, n_trials=N_TRIALS, timeout=TIMEOUT_PER_TRIAL)
tuning_time = time.time() - start_tuning_time

print(f"\n{'='*70}")
print(f"✅ Optuna 튜닝 완료!")
print(f"{'='*70}")

# ============================================================================
# 15. 최적 파라미터 확인
# ============================================================================
print("\n" + "="*70)
print("14. 최적 하이퍼파라미터")
print("="*70)

best_trial = study.best_trial

print(f"\n🏆 최고 성능 Trial: #{best_trial.number}")
print(f"   검증 정확도: {best_trial.value:.4f} ({best_trial.value*100:.2f}%)")
print(f"\n📊 최적 하이퍼파라미터:")
for param_name, param_value in best_trial.params.items():
    print(f"  • {param_name}: {param_value}")

best_batch_size = best_trial.params['batch_size']
best_epochs = best_trial.params['epochs']
best_learning_rate = best_trial.params['learning_rate']
best_max_length = best_trial.params['max_length']

# ============================================================================
# 16. 모든 Trial 결과 요약
# ============================================================================
print(f"\n{'='*70}")
print("15. 모든 Trial 결과")
print(f"{'='*70}\n")

trials_df = study.trials_dataframe()
print(f"총 시도 횟수: {len(study.trials)}")
print(f"성공한 Trial: {len([t for t in study.trials if t.state == TrialState.COMPLETE])}")
print(f"Pruned Trial: {len([t for t in study.trials if t.state == TrialState.PRUNED])}")

print(f"\n최상위 5개 Trial:")
print(trials_df[['number', 'value', 'params_batch_size', 'params_epochs',
                 'params_learning_rate', 'params_max_length']].sort_values('value', ascending=False).head(5).to_string())

# ============================================================================
# 17. 최적 파라미터로 최종 모델 학습 (클래스 가중치 적용)
# ============================================================================
print(f"\n{'='*70}")
print("16. 최적 파라미터로 최종 모델 학습 (클래스 가중치 적용)")
print(f"{'='*70}\n")

print(f"📝 학습 설정:")
print(f"  • Batch Size: {best_batch_size}")
print(f"  • Epochs: {best_epochs}")
print(f"  • Learning Rate: {best_learning_rate:.2e}")
print(f"  • Max Length: {best_max_length}")
print(f"  • 클래스 가중치: 적용 ✅\n")

# Dataset 생성
final_train_dataset = ConversationDataset(
    X_train, tokenizer, labels=y_train, max_length=best_max_length
)
final_val_dataset = ConversationDataset(
    X_val, tokenizer, labels=y_val, max_length=best_max_length
)

final_train_loader = DataLoader(final_train_dataset, batch_size=best_batch_size, shuffle=True)
final_val_loader = DataLoader(final_val_dataset, batch_size=best_batch_size)

# 최종 모델 생성
final_model = create_model(len(label_encoder.classes_))
final_optimizer = AdamW(final_model.parameters(), lr=best_learning_rate)

# 클래스 가중치가 적용된 손실 함수
final_criterion = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)

# 최종 학습
print("🔄 최종 모델 학습 중...\n")

best_final_accuracy = 0
training_history_final = []

for epoch in range(best_epochs):
    final_model.train()
    total_loss = 0
    correct = 0
    total = 0

    pbar = tqdm(final_train_loader, desc=f"Epoch {epoch+1}/{best_epochs}")

    for batch in pbar:
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        outputs = final_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # 클래스 가중치 적용된 손실
        loss = final_criterion(outputs.logits, labels)

        final_optimizer.zero_grad()
        loss.backward()
        final_optimizer.step()

        total_loss += loss.item()
        predictions = torch.argmax(outputs.logits, dim=1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

        pbar.set_postfix({
            'loss': f'{total_loss/len(pbar):.4f}',
            'acc': f'{correct/total:.4f}'
        })

    train_avg_loss = total_loss / len(final_train_loader)
    train_accuracy = correct / total

    # 검증
    final_model.eval()
    val_preds = []
    val_labels = []

    pbar_val = tqdm(final_val_loader, desc=f"Epoch {epoch+1}/{best_epochs} - Val", leave=False)

    with torch.no_grad():
        for batch in pbar_val:
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)

            outputs = final_model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=1)

            val_preds.extend(predictions.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(val_labels, val_preds)
    val_f1 = f1_score(val_labels, val_preds, average='weighted', zero_division=0)

    training_history_final.append({
        'epoch': epoch + 1,
        'train_loss': train_avg_loss,
        'train_acc': train_accuracy,
        'val_acc': val_accuracy,
        'val_f1': val_f1
    })

    print(f"✓ Epoch {epoch+1}/{best_epochs} - Train Loss: {train_avg_loss:.4f} | Train Acc: {train_accuracy:.4f} | Val Acc: {val_accuracy:.4f} | Val F1: {val_f1:.4f}")

    if val_accuracy > best_final_accuracy:
        best_final_accuracy = val_accuracy

print(f"\n✓ 최종 모델 학습 완료!")
print(f"  최고 검증 정확도: {best_final_accuracy:.4f} ({best_final_accuracy*100:.2f}%)\n")

# ============================================================================
# 18. 테스트 데이터 로드 및 전처리
# ============================================================================
print("="*70)
print("17. 테스트 데이터 로드 및 전처리")
print("="*70)

df_test = pd.read_csv(TEST_PATH)
print(f"✓ 원본 테스트 데이터: {len(df_test):,}개")

# 테스트 데이터도 전처리 (라벨 없음)
df_test['conversation'] = df_test['conversation'].apply(clean_text)
print(f"✓ 텍스트 정제 완료\n")

# ============================================================================
# 19. 테스트 Dataset 및 DataLoader 생성
# ============================================================================
print("="*70)
print("18. 테스트 Dataset 생성")
print("="*70)

test_dataset = ConversationDataset(
    df_test['conversation'].values,
    tokenizer,
    labels=None,
    max_length=best_max_length
)
test_loader = DataLoader(test_dataset, batch_size=best_batch_size)

print(f"✓ 테스트 DataLoader 생성 완료\n")

# ============================================================================
# 20. 테스트 데이터 분류 (신뢰도 3단계 + Low→일반대화)
# ============================================================================
print("="*70)
print("19. 테스트 데이터 분류 (최적화된 신뢰도 임계값)")
print("="*70 + "\n")

print("📊 신뢰도 기반 분류 설정 (최적화됨):")
print(f"  • High: ≥ {CONFIDENCE_THRESHOLD_HIGH} - 예측 그대로 사용")
print(f"  • Mid: {CONFIDENCE_THRESHOLD_MID} ~ {CONFIDENCE_THRESHOLD_HIGH} - 예측 그대로 사용")
print(f"  • Low: < {CONFIDENCE_THRESHOLD_MID} - '일반 대화'로 재분류")
print(f"  • Top-K 예측 개수: {TOP_K_PREDICTIONS}\n")

# 테스트 데이터 분류
final_model.eval()

predictions = []
original_predictions = []
confidences = []
confidence_levels = []
reclassified_flags = []
top_k_predictions_list = []
top_k_confidences_list = []

print("🔄 분류 중...\n")

with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)

        outputs = final_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)

        # Top-1 예측
        pred_ids = torch.argmax(logits, dim=1)
        pred_classes = [label_encoder.classes_[pid] for pid in pred_ids.cpu().numpy()]
        conf_scores = torch.max(probs, dim=1)[0].cpu().numpy()

        # Top-K 예측
        if USE_TOP_K:
            top_k_probs, top_k_indices = torch.topk(probs, k=min(TOP_K_PREDICTIONS, probs.size(1)), dim=1)

            for i in range(len(top_k_indices)):
                top_k_classes = [label_encoder.classes_[idx] for idx in top_k_indices[i].cpu().numpy()]
                top_k_confs = top_k_probs[i].cpu().numpy()

                top_k_predictions_list.append(top_k_classes)
                top_k_confidences_list.append(top_k_confs)
        else:
            top_k_predictions_list.extend([None] * len(pred_classes))
            top_k_confidences_list.extend([None] * len(pred_classes))

        # 신뢰도 기반 처리 (3단계)
        for pred_class, conf in zip(pred_classes, conf_scores):
            if conf >= CONFIDENCE_THRESHOLD_HIGH:
                conf_level = "High"
                final_pred = pred_class
                reclassified = False
            elif conf >= CONFIDENCE_THRESHOLD_MID:
                conf_level = "Mid"
                final_pred = pred_class
                reclassified = False
            else:
                conf_level = "Low"
                final_pred = "일반 대화"
                reclassified = True

            predictions.append(final_pred)
            original_predictions.append(pred_class)
            confidences.append(conf)
            confidence_levels.append(conf_level)
            reclassified_flags.append(reclassified)

print("\n✓ 분류 완료!\n")

# ============================================================================
# 분류 결과 통계
# ============================================================================
print("="*70)
print("📊 신뢰도 기반 분류 통계")
print("="*70 + "\n")

conf_level_counts = Counter(confidence_levels)
print("신뢰도 등급별 분포:")
for level in ["High", "Mid", "Low"]:
    if level in conf_level_counts:
        count = conf_level_counts[level]
        percentage = (count / len(predictions)) * 100
        print(f"  {level:4s}: {count:4d}개 ({percentage:5.1f}%)")

reclassified_count = sum(reclassified_flags)
print(f"\n'일반 대화'로 재분류:")
print(f"  {reclassified_count:4d}개 ({reclassified_count/len(predictions)*100:5.1f}%)")

if reclassified_count > 0:
    reclassified_original = [original_predictions[i] for i, flag in enumerate(reclassified_flags) if flag]
    original_class_counts = Counter(reclassified_original)
    print(f"\n재분류된 항목의 원본 예측 분포:")
    for class_name, count in original_class_counts.most_common():
        print(f"  {class_name:20s}: {count:3d}개")

print()

# ============================================================================
# 21. 결과 데이터프레임 생성
# ============================================================================
print("="*70)
print("20. 결과 데이터프레임 생성")
print("="*70)

df_test['prediction'] = predictions
df_test['original_prediction'] = original_predictions
df_test['confidence'] = confidences
df_test['confidence_level'] = confidence_levels
df_test['reclassified_to_normal'] = reclassified_flags

if USE_TOP_K:
    df_test['top_k_predictions'] = [
        ' | '.join([f"{cls}({conf:.3f})" for cls, conf in zip(preds, confs)])
        if preds is not None else ''
        for preds, confs in zip(top_k_predictions_list, top_k_confidences_list)
    ]

    for k in range(TOP_K_PREDICTIONS):
        df_test[f'pred_rank_{k+1}'] = [
            preds[k] if preds is not None and k < len(preds) else ''
            for preds in top_k_predictions_list
        ]
        df_test[f'conf_rank_{k+1}'] = [
            f"{confs[k]:.4f}" if confs is not None and k < len(confs) else ''
            for confs in top_k_confidences_list
        ]

print(f"✓ 결과 데이터프레임 생성 완료\n")

# ============================================================================
# 22. 결과 저장
# ============================================================================
print("="*70)
print("21. 결과 저장 (Google Drive)")
print("="*70)

OUTPUT_PATH_DETAILED = '/content/drive/My Drive/predictions_detailed.csv'
df_test.to_csv(OUTPUT_PATH_DETAILED, index=False, encoding='utf-8')
print(f"✓ 상세 버전 저장: predictions_detailed.csv")

OUTPUT_PATH_SIMPLE = '/content/drive/My Drive/predictions_simple.csv'
df_test[['idx', 'conversation', 'prediction', 'confidence']].to_csv(
    OUTPUT_PATH_SIMPLE, index=False, encoding='utf-8'
)
print(f"✓ 간단 버전 저장: predictions_simple.csv")

OUTPUT_PATH_RECLASSIFIED = '/content/drive/My Drive/predictions_reclassified.csv'
df_reclassified = df_test[df_test['reclassified_to_normal'] == True]
df_reclassified.to_csv(OUTPUT_PATH_RECLASSIFIED, index=False, encoding='utf-8')
print(f"✓ 재분류 버전 저장: predictions_reclassified.csv ({len(df_reclassified)}개)")

OUTPUT_PATH_TRUSTED = '/content/drive/My Drive/predictions_trusted.csv'
df_trusted = df_test[df_test['reclassified_to_normal'] == False]
df_trusted.to_csv(OUTPUT_PATH_TRUSTED, index=False, encoding='utf-8')
print(f"✓ 신뢰 버전 저장: predictions_trusted.csv ({len(df_trusted)}개)")

print()

# ============================================================================
# 신뢰도 시각화
# ============================================================================
def visualize_confidence_distribution():
    """신뢰도 분포 시각화"""
    plt.rcParams['font.family'] = 'DejaVu Sans'
    plt.rcParams['axes.unicode_minus'] = False

    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # 1. 신뢰도 히스토그램
    axes[0, 0].hist(df_test['confidence'], bins=50, color='skyblue', edgecolor='black')
    axes[0, 0].axvline(CONFIDENCE_THRESHOLD_MID, color='orange', linestyle='--',
                       label=f'Mid/Low ({CONFIDENCE_THRESHOLD_MID})')
    axes[0, 0].axvline(CONFIDENCE_THRESHOLD_HIGH, color='green', linestyle='--',
                       label=f'High/Mid ({CONFIDENCE_THRESHOLD_HIGH})')
    axes[0, 0].set_title('Confidence Score Distribution', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Confidence Score')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].legend()
    axes[0, 0].grid(axis='y', alpha=0.3)

    # 2. 신뢰도 등급별 분포
    level_data = df_test['confidence_level'].value_counts()
    colors = {'High': 'green', 'Mid': 'yellow', 'Low': 'red'}
    colors_list = [colors.get(level, 'gray') for level in level_data.index]

    axes[0, 1].bar(range(len(level_data)), level_data.values, color=colors_list, edgecolor='black')
    axes[0, 1].set_xticks(range(len(level_data)))
    axes[0, 1].set_xticklabels(level_data.index, rotation=0)
    axes[0, 1].set_title('Confidence Level Distribution', fontsize=12, fontweight='bold')
    axes[0, 1].set_ylabel('Count')
    axes[0, 1].grid(axis='y', alpha=0.3)

    # 3. 최종 예측 클래스별 분포
    pred_counts = df_test['prediction'].value_counts()
    axes[1, 0].barh(range(len(pred_counts)), pred_counts.values, color='lightcoral')
    axes[1, 0].set_yticks(range(len(pred_counts)))
    axes[1, 0].set_yticklabels([str(c)[:20] for c in pred_counts.index], fontsize=8)
    axes[1, 0].set_title('Final Prediction Distribution', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('Count')
    axes[1, 0].grid(axis='x', alpha=0.3)

    # 4. 재분류 여부
    reclass_counts = df_test['reclassified_to_normal'].value_counts()
    colors_reclass = ['green', 'orange']
    labels_reclass = ['Original Prediction', 'Reclassified to Normal']
    axes[1, 1].pie(reclass_counts.values, labels=labels_reclass, autopct='%1.1f%%',
                   colors=colors_reclass, startangle=90)
    axes[1, 1].set_title('Reclassification Status', fontsize=12, fontweight='bold')

    plt.tight_layout()
    plt.savefig('/content/drive/My Drive/confidence_analysis.png', dpi=300, bbox_inches='tight')
    print("✓ 신뢰도 분석 시각화 저장: confidence_analysis.png\n")

try:
    visualize_confidence_distribution()
except Exception as e:
    print(f"⚠️ 시각화 중 오류 발생: {e}\n")

# ============================================================================
# 23. 최종 완료 메시지
# ============================================================================
print("\n\n" + "█"*70)
print("█" + " "*68 + "█")
print("█" + "  ✅ 모든 작업이 완료되었습니다!".center(68) + "█")
print("█" + " "*68 + "█")
print("█"*70)

print("\n📁 저장된 파일 정보:")
print(f"\n  📊 예측 결과 파일 (4가지 버전):")
print(f"     1. predictions_detailed.csv - 전체 상세 ({len(df_test)}개)")
print(f"     2. predictions_simple.csv - 최종 예측만 ({len(df_test)}개)")
print(f"     3. predictions_reclassified.csv - '일반 대화' 재분류 ({len(df_reclassified)}개)")
print(f"     4. predictions_trusted.csv - High+Mid 신뢰도 ({len(df_trusted)}개)")

print(f"\n  📈 시각화:")
print(f"     5. confidence_analysis.png - 신뢰도 분석")

print(f"\n🎯 최적 하이퍼파라미터:")
print(f"  • Batch Size: {best_batch_size}")
print(f"  • Epochs: {best_epochs}")
print(f"  • Learning Rate: {best_learning_rate:.2e}")
print(f"  • Max Length: {best_max_length}")

print(f"\n📈 최종 모델 성능:")
print(f"  • 검증 정확도: {best_final_accuracy:.4f} ({best_final_accuracy*100:.2f}%)")

print(f"\n🎯 신뢰도 기반 분류 통계:")
print(f"  • High (≥{CONFIDENCE_THRESHOLD_HIGH}): {conf_level_counts.get('High', 0)}개 ({conf_level_counts.get('High', 0)/len(predictions)*100:.1f}%)")
print(f"  • Mid ({CONFIDENCE_THRESHOLD_MID}~{CONFIDENCE_THRESHOLD_HIGH}): {conf_level_counts.get('Mid', 0)}개 ({conf_level_counts.get('Mid', 0)/len(predictions)*100:.1f}%)")
print(f"  • Low (<{CONFIDENCE_THRESHOLD_MID}): {conf_level_counts.get('Low', 0)}개 ({conf_level_counts.get('Low', 0)/len(predictions)*100:.1f}%) → '일반 대화' 재분류")

print(f"\n🔥 적용된 개선사항:")
print(f"  ✅ 완벽한 전처리 (BOM, 공백, 따옴표, 길이)")
print(f"  ✅ 클래스 가중치 적용 (불균형 해결)")
print(f"  ✅ 신뢰도 임계값 최적화 (0.8→0.6, 0.9→0.85)")
print(f"  ✅ 데이터 품질 검증")
print(f"  ✅ 토크나이저 호환성 체크")

print(f"\n⏱️  수행 시간:")
print(f"  • Optuna 튜닝: {tuning_time:.1f}초 ({tuning_time/60:.1f}분)")

print(f"\n💾 모든 파일이 Google Drive에 저장되었습니다!")
print("\n" + "█"*70 + "\n")

01. GPU 메모리 최적화 설정 완료

02. 라이브러리 설치
✓ 모든 라이브러리 설치 완료

03. 기본 설정
✓ 모델: beomi/kcbert-base
✓ 디바이스: cuda
✓ Optuna 시도 횟수: 5
✓ Validation 비율: 20.0%

✓ 신뢰도 임계값 (3단계) - 최적화:
  • High: ≥ 0.85
  • Mid: 0.6 ~ 0.85
  • Low: < 0.6 → '일반 대화'로 재분류

✓ 전처리 설정:
  • 최소 길이: 10자
  • 최대 길이: 1500자

💾 GPU 정보:
  • GPU: Tesla T4
  • 총 메모리: 14.74 GB

04. 전처리 함수 정의
✓ 전처리 함수 정의 완료

05. Dataset 클래스 정의
✓ Dataset 클래스 정의 완료

06. 학습 데이터 로드 및 완벽한 전처리
✓ 원본 데이터: 4,950개

🧹 데이터 전처리 시작...
  원본 데이터: 4,950개
  ✓ 텍스트 정제 중 (BOM, 공백, 따옴표, 특수문자)...
  ✓ 중복 제거: 105개

  ✅ 전처리 완료!
  최종 데이터: 4,845개 (유지율: 97.9%)
  총 제거: 105개

📊 클래스 분포:
class
기타 괴롭힘 대화      1011
일반 대화           999
갈취 대화           973
직장 내 괴롭힘 대화     970
협박 대화           892
Name: count, dtype: int64

⚖️  클래스 균형도 분석:
  최대 클래스: 1,011개
  최소 클래스: 892개
  불균형 비율: 1.13:1
  ✅ 균형 잡힌 데이터셋

07. 라벨 인코딩
✓ 클래스 개수: 5

클래스 매핑:
  0: 갈취 대화                -   973개 ( 20.1%)
  1: 기타 괴롭힘 대화            - 1,011개 ( 20.9%)
  2: 일반 대화                -   999개 ( 20.6%)
  3: 직장 내 괴롭힘 대화          -   9

Token indices sequence length is longer than the specified maximum sequence length for this model (321 > 300). Running this sequence through the model will result in indexing errors


✓ 토크나이저 로드 완료
✓ 모델 최대 위치 임베딩: 300
✓ 사용 가능한 최대 길이: 300

🔧 토크나이저 호환성 체크 (샘플 1000개)...


[I 2025-11-10 17:43:49,016] A new study created in memory with name: no-name-af7d57df-d90f-4f27-b080-909740ada74d


  • 평균 토큰 길이: 110.3
  • 최대 토큰 길이: 328
  • MAX_LENGTH(300) 초과: 3개 (0.3%)
  ⚠️ 일부 데이터가 MAX_LENGTH 초과. 잘림 현상 발생 가능

11. 모델 로드 함수 정의 완료

12. 목적 함수 정의 완료 (클래스 가중치 적용)

13. Optuna 하이퍼파라미터 튜닝 시작

Trial 1

📊 제안된 하이퍼파라미터:
  • Batch Size: 16
  • Epochs: 5
  • Learning Rate: 2.62e-05
  • Max Length: 256

💾 GPU 메모리 (Trial 시작):
  • 할당됨: 3.99 GB
  • 예약됨: 4.05 GB


...

✓ Epoch 4/5 - Train Loss: 0.0397 | Train Acc: 0.9897 | Val Acc: 0.9216 | Val F1: 0.9211


Epoch 5/5: 100%|██████████| 243/243 [02:48<00:00,  1.45it/s, loss=0.0187, acc=0.9948]


✓ Epoch 5/5 - Train Loss: 0.0187 | Train Acc: 0.9948 | Val Acc: 0.9112 | Val F1: 0.9101

✓ 최종 모델 학습 완료!
  최고 검증 정확도: 0.9216 (92.16%)

17. 테스트 데이터 로드 및 전처리
✓ 원본 테스트 데이터: 500개
✓ 텍스트 정제 완료

18. 테스트 Dataset 생성
✓ 테스트 DataLoader 생성 완료

19. 테스트 데이터 분류 (최적화된 신뢰도 임계값)

📊 신뢰도 기반 분류 설정 (최적화됨):
  • High: ≥ 0.85 - 예측 그대로 사용
  • Mid: 0.6 ~ 0.85 - 예측 그대로 사용
  • Low: < 0.6 - '일반 대화'로 재분류
  • Top-K 예측 개수: 3

🔄 분류 중...



100%|██████████| 32/32 [00:07<00:00,  4.48it/s]



✓ 분류 완료!

📊 신뢰도 기반 분류 통계

신뢰도 등급별 분포:
  High:  447개 ( 89.4%)
  Mid :   35개 (  7.0%)
  Low :   18개 (  3.6%)

'일반 대화'로 재분류:
    18개 (  3.6%)

재분류된 항목의 원본 예측 분포:
  갈취 대화               :   6개
  협박 대화               :   5개
  직장 내 괴롭힘 대화         :   5개
  기타 괴롭힘 대화           :   2개

20. 결과 데이터프레임 생성
✓ 결과 데이터프레임 생성 완료

21. 결과 저장 (Google Drive)
✓ 상세 버전 저장: predictions_detailed.csv
✓ 간단 버전 저장: predictions_simple.csv
✓ 재분류 버전 저장: predictions_reclassified.csv (18개)
✓ 신뢰 버전 저장: predictions_trusted.csv (482개)

✓ 신뢰도 분석 시각화 저장: confidence_analysis.png



██████████████████████████████████████████████████████████████████████
█                                                                    █
█                          ✅ 모든 작업이 완료되었습니다!                         █
█                                                                    █
██████████████████████████████████████████████████████████████████████

📁 저장된 파일 정보:

  📊 예측 결과 파일 (4가지 버전):
     1. predictions_detailed.csv - 전체 상세 (500개)
     2. predi

In [7]:
# ============================================================================
# 아웃풋 파일을 통한 결과 요약
# ============================================================================
import pandas as pd

df_pred = pd.read_csv('/content/drive/My Drive/predictions_detailed.csv')

print("\n" + "🎯 예측 결과 요약".center(50, "="))
print(f"\n총 예측: {len(df_pred):,}개\n")

print("📊 클래스별 분포:")

if 'confidence' in df_pred.columns:
    print(f"\n📌 전체 신뢰도 요약:")
    print("-" * 50)
    print(f"평균: {df_pred['confidence'].mean():.4f}")
    print(f"최소: {df_pred['confidence'].min():.4f}")
    print(f"최대: {df_pred['confidence'].max():.4f}")

    high = (df_pred['confidence'] >= 0.9).sum()
    mid = ((df_pred['confidence'] >= 0.7) & (df_pred['confidence'] < 0.9)).sum()
    low = (df_pred['confidence'] < 0.7).sum()

    print(f"\n높은 신뢰도 (≥0.9):   {high:>5}개 ({high/len(df_pred)*100:>5.1f}%)")
    print(f"중간 신뢰도 (0.7~0.9): {mid:>5}개 ({mid/len(df_pred)*100:>5.1f}%)")
    print(f"낮은 신뢰도 (<0.7):   {low:>5}개 ({low/len(df_pred)*100:>5.1f}%)")

    # ✨ 클래스별 상세 신뢰도 분포
    print(f"\n📌 클래스별 신뢰도 상세 분석:")
    print("=" * 90)

    # 평균 신뢰도 기준으로 정렬
    class_stats = []
    for class_name in df_pred['prediction'].unique():
        class_data = df_pred[df_pred['prediction'] == class_name]
        avg_conf = class_data['confidence'].mean()
        class_stats.append((class_name, avg_conf))

    class_stats.sort(key=lambda x: x[1], reverse=True)

    for class_name, _ in class_stats:
        class_data = df_pred[df_pred['prediction'] == class_name]

        count = len(class_data)
        avg_conf = class_data['confidence'].mean()
        min_conf = class_data['confidence'].min()
        max_conf = class_data['confidence'].max()
        std_conf = class_data['confidence'].std()

        high_class = (class_data['confidence'] >= 0.9).sum()
        mid_class = ((class_data['confidence'] >= 0.7) & (class_data['confidence'] < 0.9)).sum()
        low_class = (class_data['confidence'] < 0.7).sum()

        high_pct = high_class / count * 100 if count > 0 else 0
        mid_pct = mid_class / count * 100 if count > 0 else 0
        low_pct = low_class / count * 100 if count > 0 else 0

        print(f"\n🔹 {class_name} (총 {count}개)")
        print(f"   평균: {avg_conf:.4f} | 최소: {min_conf:.4f} | 최대: {max_conf:.4f} | 표준편차: {std_conf:.4f}")
        print(f"   높음 (≥0.9):   {high_class:>4}개 ({high_pct:>5.1f}%)", end="")
        print(f" {'█' * int(high_pct / 5)}")
        print(f"   중간 (0.7~0.9): {mid_class:>4}개 ({mid_pct:>5.1f}%)", end="")
        print(f" {'█' * int(mid_pct / 5)}")
        print(f"   낮음 (<0.7):   {low_class:>4}개 ({low_pct:>5.1f}%)", end="")
        print(f" {'█' * int(low_pct / 5)}")

    print("\n" + "=" * 90)

print("\n" + "="*70)


====================🎯 예측 결과 요약====================

총 예측: 500개

📊 클래스별 분포:

📌 전체 신뢰도 요약:
--------------------------------------------------
평균: 0.9502
최소: 0.3171
최대: 0.9994

높은 신뢰도 (≥0.9):     436개 ( 87.2%)
중간 신뢰도 (0.7~0.9):    30개 (  6.0%)
낮은 신뢰도 (<0.7):      34개 (  6.8%)

📌 클래스별 신뢰도 상세 분석:

🔹 직장 내 괴롭힘 대화 (총 140개)
   평균: 0.9774 | 최소: 0.6149 | 최대: 0.9994 | 표준편차: 0.0653
   높음 (≥0.9):    129개 ( 92.1%) ██████████████████
   중간 (0.7~0.9):    7개 (  5.0%) █
   낮음 (<0.7):      4개 (  2.9%) 

🔹 협박 대화 (총 90개)
   평균: 0.9761 | 최소: 0.6045 | 최대: 0.9993 | 표준편차: 0.0709
   높음 (≥0.9):     83개 ( 92.2%) ██████████████████
   중간 (0.7~0.9):    4개 (  4.4%) 
   낮음 (<0.7):      3개 (  3.3%) 

🔹 갈취 대화 (총 120개)
   평균: 0.9731 | 최소: 0.6047 | 최대: 0.9990 | 표준편차: 0.0706
   높음 (≥0.9):    112개 ( 93.3%) ██████████████████
   중간 (0.7~0.9):    4개 (  3.3%) 
   낮음 (<0.7):      4개 (  3.3%) 

🔹 기타 괴롭힘 대화 (총 130개)
   평균: 0.9498 | 최소: 0.6052 | 최대: 0.9992 | 표준편차: 0.0885
   높음 (≥0.9):    110개 ( 84.6%) ████████████████
   중간 (0.7~

요약 및 개선안
1) 어제 최고 평점이 나왔던 개선안이 토요일 새벽 버전의 아웃풋인데 이후 계속 수정되고 왔기 때문에, 현재 어떤 상태에서 그런 결과물이 나왔는지 그대로 재현되고 있지 않음 (안타깝습니다.) 그러나, 개선방향과 포텐셜은 명확하게 보임
1) 일반 대화의 다양성 부여 및 강화. 이를 위해 일반 대화 생성기를 만들어 볼 생각임. 그래야 여러가지 로직을 반영한 일반 대화를 생성하고 통합해야 풍부한 정보를 담은 일반 대화 합성이 가능될 것으로 예측됨
2) 분류시 확률에 따라 실행하는데, 이때 확률에 따른 컨피던스를 가지고 분류 함. 불균형적인 분류 성능을 개선하기 위하여 낮은 컨피던스를 가진 경우 일반 대화로 분류하는 방식 적용 (혹은  임계값 방식 반영)해 보고자 함. 추가 코드 블럭에 예시를 제시함

# **추가 블럭**

In [11]:
#제출포맷 변경 - path 잘 지정해야합니다.
# 1. CSV 파일 읽기
df = pd.read_csv('/content/drive/My Drive/predictions_detailed.csv') #패스 지정!

print("원본 파일 확인:")
print(f"행 수: {len(df)}")
print(f"열: {df.columns.tolist()}")
print("\n처음 5행:")
print(df.head())

# 2. idx와 prediction 열만 선택
df_selected = df[['idx', 'prediction']].copy()

# 3. prediction 열을 class로 이름 변경
df_selected.rename(columns={'prediction': 'class'}, inplace=True)

# 4. 매핑 딕셔너리 생성
class_mapping = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
    '일반 대화': 4
}

# 5. prediction 값을 숫자로 변환
# Note: The column is now named 'class', so we apply the mapping to that column.
df_selected['class'] = df_selected['class'].map(class_mapping)


# 6. 결과 확인
print("\n\n변환 후 파일 확인:")
print(f"행 수: {len(df_selected)}")
print(f"열: {df_selected.columns.tolist()}")
print(f"데이터 모양: {df_selected.shape}") # Add this line to show the shape
print("\n처음 10行:")
print(df_selected.head(10))

# 7. 변환된 값 확인
print("\n\n변환 통계:")
print(df_selected['class'].value_counts().sort_index())

# 8. CSV로 저장 (예: 구글 드라이브 My Drive 루트에 저장)
output_path = '/content/drive/My Drive/prediction_final.csv'  # 저장할 경로 지정
df_selected.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n변환된 파일이 저장되었습니다: {output_path}")


원본 파일 확인:
행 수: 500
열: ['idx', 'conversation', 'prediction', 'original_prediction', 'confidence', 'confidence_level', 'reclassified_to_normal', 'top_k_predictions', 'pred_rank_1', 'conf_rank_1', 'pred_rank_2', 'conf_rank_2', 'pred_rank_3', 'conf_rank_3']

처음 5행:
     idx                                       conversation   prediction  \
0  t_000  아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...        갈취 대화   
1  t_001  우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? 그...  직장 내 괴롭힘 대화   
2  t_002  너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...  직장 내 괴롭힘 대화   
3  t_003  이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...    기타 괴롭힘 대화   
4  t_004  아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...        일반 대화   

  original_prediction  confidence confidence_level  reclassified_to_normal  \
0               갈취 대화    0.998091             High                   False   
1         직장 내 괴롭힘 대화    0.999237             High                   False   
2         직장 내 괴롭힘 대화    0.843358              

In [ ]:
# 데이터 임베딩 구조 및 테스트, 테스트 블럭
# ============================================================================
# 🔍 모델 구조 진단
# ============================================================================
print("="*70)
print("모델 구조 진단")
print("="*70)

from transformers import AutoConfig, AutoModelForSequenceClassification

# 1. Config 확인
config = AutoConfig.from_pretrained('beomi/kcbert-base')
print(f"\n📋 모델 Config:")
print(f"  • max_position_embeddings: {config.max_position_embeddings}")
print(f"  • hidden_size: {config.hidden_size}")
print(f"  • vocab_size: {config.vocab_size}")

# 2. 토크나이저 확인
tokenizer = AutoTokenizer.from_pretrained('beomi/kcbert-base')
print(f"\n🔤 토크나이저:")
print(f"  • model_max_length: {tokenizer.model_max_length}")
print(f"  • vocab_size: {len(tokenizer)}")

# 3. 실제 모델 로드 후 임베딩 크기 확인
model = AutoModelForSequenceClassification.from_pretrained(
    'beomi/kcbert-base',
    num_labels=len(label_encoder.classes_)
)

print(f"\n🧠 모델 레이어:")
print(f"  • Position Embeddings: {model.bert.embeddings.position_embeddings.weight.shape}")
print(f"  • Token Embeddings: {model.bert.embeddings.word_embeddings.weight.shape}")
print(f"  • Classifier: {model.classifier.weight.shape}")

# 4. 테스트 입력으로 확인
test_text = "테스트 문장입니다."
# Use the smaller of tokenizer's model_max_length and config's max_position_embeddings
max_seq_length = min(tokenizer.model_max_length, config.max_position_embeddings)

test_encoding = tokenizer(test_text, max_length=max_seq_length, padding='max_length',
                          truncation=True, return_tensors='pt')
print(f"\n🧪 테스트 입력:")
print(f"  • input_ids shape: {test_encoding['input_ids'].shape}")
print(f"  • attention_mask shape: {test_encoding['attention_mask'].shape}")

try:
    with torch.no_grad():
        output = model(**test_encoding)
    print(f"  • 모델 출력: ✅ 정상")
except Exception as e:
    print(f"  • 모델 출력: ❌ 에러 - {str(e)}")

print("="*70 + "\n")

모델 구조 진단

📋 모델 Config:
  • max_position_embeddings: 300
  • hidden_size: 768
  • vocab_size: 30000

🔤 토크나이저:
  • model_max_length: 300
  • vocab_size: 30000


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🧠 모델 레이어:
  • Position Embeddings: torch.Size([300, 768])
  • Token Embeddings: torch.Size([30000, 768])
  • Classifier: torch.Size([5, 768])

🧪 테스트 입력:
  • input_ids shape: torch.Size([1, 300])
  • attention_mask shape: torch.Size([1, 300])
  • 모델 출력: ✅ 정상



In [ ]:
# 컨피던스 기반 분류
# 낮은 confidence를 가진 갈취 → 일반으로 재분류
def reclassify_low_confidence(df, confidence_threshold=0.7):
    """
    갈취로 분류됐지만 확신이 낮으면 일반으로 변경
    """
    df_copy = df.copy()

    # 갈취인데 confidence가 낮은 경우
    mask = (df_copy['prediction'] == '갈취 대화') & (df_copy['confidence'] < confidence_threshold)
    df_copy.loc[mask, 'prediction'] = '일반 대화'

    print(f"재분류된 샘플 수: {mask.sum()}개")
    return df_copy

# 사용
df_improved = reclassify_low_confidence(df, confidence_threshold=0.75)

In [ ]:
# 임계값 부여 방식 분류
# 현재: 가장 높은 확률의 클래스를 선택
predicted_class = np.argmax(predictions)

# 개선: 임계값 조정
def predict_with_threshold(predictions, thresholds):
    """
    thresholds: {0: 0.6, 1: 0.5, 2: 0.5, 3: 0.5, 4: 0.3}
    갈취(1)는 높은 확률 필요, 일반(4)은 낮은 확률로도 허용
    """
    max_class = np.argmax(predictions)
    max_prob = predictions[max_class]

    # 일반 대화 우선 검토
    if predictions[4] > thresholds[4]:  # 일반이 30% 이상이면
        return 4

    # 갈취는 엄격하게
    if max_class == 1 and max_prob < thresholds[1]:  # 갈취가 50% 미만이면
        return 4  # 일반으로 분류

    return max_class

# 사용 예시
thresholds = {
    0: 0.6,  # 협박
    1: 0.7,  # 갈취 (엄격하게!)
    2: 0.6,  # 직장 내 괴롭힘
    3: 0.6,  # 기타 괴롭힘
    4: 0.2   # 일반 (관대하게!)
}